In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [2]:

df = pd.read_csv('./spambase/spambase.data', delimiter=';')
df.head()


,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam_class
0,0,64,64,0,32,0,0,0,0,0,...,0,0,0,778,0,0,3756,61,278,1
1,21,28,5,0,14,28,21,7,0,94,...,0,132,0,372,18,48,5114,101,1028,1
2,6,0,71,0,123,19,19,12,64,25,...,1,143,0,276,184,1,9821,485,2259,1
3,0,0,0,0,63,0,31,63,31,63,...,0,137,0,137,0,0,3537,40,191,1
4,0,0,0,0,63,0,31,63,31,63,...,0,135,0,135,0,0,3537,40,191,1


In [3]:
df.shape

(4601, 58)